# 第 5 步: 获取未来 CAMS AOD 预报数据

此 Notebook 负责从哥白尼大气监测服务 (CAMS) 下载未来的**气溶胶光学厚度 (AOD)** 预报数据。

AOD 是影响天空颜色和通透度的关键因素，高 AOD（尤其是由特定类型的气溶胶引起时）可以显著增强日出和日落的色彩。这是计算“火烧云指数”的重要补充数据。

**核心逻辑**:
1. CAMS 每天在 00z 和 12z (UTC时间) 发布两次全球预报。
2. 智能地找到当前可用的、最新的那一次预报（CAMS 数据发布延迟较长，需要更大的安全边际）。
3. 根据我们关心的未来日出/日落时间，计算出相对于运行周期的“预报时效”列表。
4. 构建下载请求，**仅下载我们关心区域和时间点**的 GRIB 数据，以节省时间和空间。

In [1]:
import logging
import json
import cdsapi
import zipfile
from datetime import datetime, timedelta, timezone
from pathlib import Path
from zoneinfo import ZoneInfo
from typing import Tuple, Dict

# --- 1. 从我们的配置文件中导入所有常量 ---
# 确保您的项目路径已正确设置，以便能够找到 chromasky_toolkit
try:
    from chromasky_toolkit import config
except ImportError:
    # 如果直接运行脚本，可能需要手动将 src 目录添加到 sys.path
    import sys
    # 根据此脚本的位置调整路径
    project_root = Path(__file__).resolve().parent.parent
    src_path = project_root / 'src'
    if str(src_path) not in sys.path:
        sys.path.insert(0, str(src_path))
    from chromasky_toolkit import config

# --- 2. 设置日志 ---
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger("CAMS_AOD_Downloader")

# --- 3. 验证导入的配置 ---
print("\n--- 使用的配置 ---")
print(f"CDS API Key Loaded: {'Yes' if config.CDS_API_KEY else 'No'}")
print(f"CAMS AOD Data Directory: {config.CAMS_AOD_DATA_DIR}")
print(f"CAMS Dataset Name: {config.CAMS_DATASET_NAME}")
print(f"CAMS AOD Variables: {config.CAMS_AOD_VARIABLES}")
print(f"Extraction Area (North): {config.AREA_EXTRACTION['north']}")

⚠️ Config: 未找到 .env 文件于 C:\Users\zhang\Documents\Code\chromasky-toolkit\src\.env

--- 使用的配置 ---
CDS API Key Loaded: Yes
CAMS AOD Data Directory: C:\Users\zhang\Documents\Code\chromasky-toolkit\src\data\raw\cams_aod
CAMS Dataset Name: cams-global-atmospheric-composition-forecasts
CAMS AOD Variables: ['total_aerosol_optical_depth_550nm']
Extraction Area (North): 54.0


In [2]:

def _find_latest_available_cams_run() -> Tuple[datetime.date, str] | None:
    """
    智能判断当前可用的最新 CAMS 运行周期 (00z 或 12z)。
    CAMS 数据通常有较长的延迟，我们设置一个安全边际。
    """
    logger.info("--- [CAMS] 正在寻找最新的可用运行周期... ---")
    now_utc = datetime.now(timezone.utc)
    # CAMS 数据延迟通常比 GFS 长，设置一个 8-9 小时的安全边际
    safe_margin = timedelta(hours=9)
    
    # 按时间倒序检查过去两天内的所有可能运行周期
    potential_runs = [
        (now_utc.date(), "12:00"),
        (now_utc.date(), "00:00"),
        (now_utc.date() - timedelta(days=1), "12:00"),
        (now_utc.date() - timedelta(days=1), "00:00"),
    ]

    for run_date, run_hour_str in potential_runs:
        run_time_utc = datetime.strptime(f"{run_date.strftime('%Y-%m-%d')} {run_hour_str}", "%Y-%m-%d %H:%M").replace(tzinfo=timezone.utc)
        if (now_utc - run_time_utc) >= safe_margin:
            logger.info(f"✅ 找到最新的可用运行周期: {run_date.strftime('%Y-%m-%d')} {run_hour_str} UTC")
            return run_date, run_hour_str
            
    logger.error("❌ 在过去48小时内未能找到任何满足安全边际的 CAMS 运行周期。")
    return None


def _get_future_target_times() -> Dict[str, datetime]:
    """根据配置，计算未来1-2天我们关心的日出日落事件的 UTC 时间。"""
    local_tz = ZoneInfo(config.LOCAL_TZ)
    now_local = datetime.now(local_tz)
    today = now_local.date()
    tomorrow = today + timedelta(days=1)
    future_events = {}
    all_times = config.SUNRISE_EVENT_TIMES + config.SUNSET_EVENT_TIMES
    
    for t_str in all_times:
        event_time = datetime.strptime(t_str, '%H:%M').time()
        # 检查今天的事件是否仍在未来
        today_event_dt = datetime.combine(today, event_time, tzinfo=local_tz)
        if today_event_dt > now_local:
            future_events[f"today_{t_str.replace(':', '')}"] = today_event_dt
        
        # 总是包含明天的所有事件
        tomorrow_event_dt = datetime.combine(tomorrow, event_time, tzinfo=local_tz)
        future_events[f"tomorrow_{t_str.replace(':', '')}"] = tomorrow_event_dt
        
    return {name: dt.astimezone(timezone.utc) for name, dt in sorted(future_events.items())}

In [3]:

def download_cams_aod_forecast(run_date_obj: datetime.date, run_hour_str: str) -> bool:
    """
    为给定的 CAMS 运行周期，下载覆盖所有未来目标事件的 AOD 数据。
    *** 新版本：请求 NetCDF 并处理可能的 ZIP 压缩包 ***
    """
    run_date_str = run_date_obj.strftime('%Y-%m-%d')
    logger.info(f"--- [CAMS] 开始为运行周期 {run_date_str} {run_hour_str} UTC 下载数据 ---")

    # 1. 构建输出目录和文件路径
    output_dir_name = f"{run_date_obj.strftime('%Y%m%d')}_t{run_hour_str[:2]}z"
    output_dir = config.CAMS_AOD_DATA_DIR / output_dir_name
    output_dir.mkdir(parents=True, exist_ok=True)
    
    manifest_path = output_dir / "manifest_aod.json"
    final_output_file = output_dir / "aod_forecast.nc" # 最终文件名后缀为 .nc
    temp_download_path = output_dir / "temp_download" # 临时下载文件

    # 幂等性检查：如果清单和最终数据都存在，则跳过
    if manifest_path.exists() and final_output_file.exists():
        logger.info(f"✅ 数据和清单文件已在 '{output_dir}' 存在，跳过下载。")
        return True

    # 2. 动态计算需要的预报时效 (leadtime hours)
    base_run_time = datetime.strptime(f"{run_date_str} {run_hour_str}", "%Y-%m-%d %H:%M").replace(tzinfo=timezone.utc)
    target_events_utc = _get_future_target_times()
    
    leadtime_hours_set = set()
    for event_name, target_time in target_events_utc.items():
        if target_time > base_run_time:
            time_diff = target_time - base_run_time
            forecast_hour = round(time_diff.total_seconds() / 3600)
            leadtime_hours_set.add(forecast_hour)

    if not leadtime_hours_set:
        logger.warning("[CAMS] 计算后没有需要下载的未来预报时效，任务结束。")
        return False

    leadtime_hours_list = sorted([str(h) for h in leadtime_hours_set])
    logger.info(f"将为 CAMS 运行周期下载 {len(leadtime_hours_list)} 个特定预报时效的数据: {leadtime_hours_list}")

    # 3. 构建 API 请求并下载
    try:
        c = cdsapi.Client(timeout=600, quiet=False, url="https://ads.atmosphere.copernicus.eu/api", key=config.CDS_API_KEY)
        
        area_bounds = [config.AREA_EXTRACTION[k] for k in ["north", "west", "south", "east"]]
        
        request_params = {
            'date': [run_date_str], # API v2 格式简化
            'time': [run_hour_str],
            'format': 'netcdf', # 请求 netcdf 格式
            'variable': config.CAMS_AOD_VARIABLES,
            'leadtime_hour': leadtime_hours_list,
            'type': ['forecast'],
            'area': area_bounds
        }

        logger.info("正在向 Copernicus ADS 发送请求以下载区域数据...")
        # 下载到临时路径
        c.retrieve(config.CAMS_DATASET_NAME, request_params, str(temp_download_path))
        logger.info(f"✅ 临时文件已成功下载到: {temp_download_path}")

        # 4. 检查下载的是 ZIP 还是直接就是 NC，并进行处理
        if zipfile.is_zipfile(temp_download_path):
            logger.info("检测到下载文件为 ZIP 压缩包，开始解压...")
            with zipfile.ZipFile(temp_download_path, 'r') as zip_ref:
                nc_files_in_zip = [f for f in zip_ref.namelist() if f.endswith('.nc')]
                if not nc_files_in_zip:
                    raise FileNotFoundError("ZIP 包中未找到任何 .nc 文件！")
                
                # 解压第一个找到的 .nc 文件到 output_dir
                source_nc_path_str = zip_ref.extract(nc_files_in_zip[0], path=output_dir)
                source_nc_path = Path(source_nc_path_str)
                logger.info(f"已解压出 NetCDF 文件: {source_nc_path}")
                
                # 将解压出的文件重命名为我们最终想要的名字
                source_nc_path.rename(final_output_file)
                logger.info(f"已将文件重命名为: {final_output_file}")
        else:
            # 如果不是 ZIP，说明直接下载的就是 NetCDF 文件
            logger.info("检测到下载文件为 NetCDF，直接重命名。")
            temp_download_path.rename(final_output_file)
        
        # 5. *** 核心改动：创建包含详细时间信息的清单文件 ***
        local_tz = ZoneInfo(config.LOCAL_TZ)
        forecast_details = []
        # 使用整数小时列表进行计算
        for hour in sorted([int(h) for h in leadtime_hours_list]):
            forecast_time_utc = base_run_time + timedelta(hours=hour)
            forecast_time_local = forecast_time_utc.astimezone(local_tz)
            forecast_details.append({
                "leadtime_hour": hour,
                "forecast_time_utc": forecast_time_utc.isoformat(),
                "forecast_time_local": forecast_time_local.isoformat(),
            })

        manifest_content = {
            "base_run_time_utc": base_run_time.isoformat(),
            "data_file_path": str(final_output_file.relative_to(config.PROJECT_ROOT)),
            "forecasts": forecast_details # 使用新的、更详细的结构
        }
        
        with open(manifest_path, 'w') as f:
            json.dump(manifest_content, f, indent=4)
        logger.info(f"✅ AOD 数据清单已成功写入: {manifest_path}")
        return True

    except Exception as e:
        logger.error(f"❌ 下载或处理 CAMS AOD 数据时发生严重错误: {e}", exc_info=True)
        return False
    finally:
        # 6. 清理临时下载文件
        if temp_download_path.exists():
            temp_download_path.unlink()
            logger.info(f"已清理临时文件: {temp_download_path}")


In [4]:
# --- 执行 CAMS AOD 预报数据下载 ---
cams_run_info = _find_latest_available_cams_run()

if cams_run_info:
    run_date, run_hour = cams_run_info
    success = download_cams_aod_forecast(run_date, run_hour)
    if success:
        logger.info("\n🎉 CAMS AOD 预报数据下载任务成功完成！")
    else:
        logger.error("\n😢 CAMS AOD 预报数据下载任务失败。")
else:
    logger.error("未能找到可用的 CAMS 运行周期，今日 AOD 预报下载任务无法执行。")

2025-08-09 22:33:29,690 - INFO - --- [CAMS] 正在寻找最新的可用运行周期... ---
2025-08-09 22:33:29,691 - INFO - ✅ 找到最新的可用运行周期: 2025-08-09 00:00 UTC
2025-08-09 22:33:29,692 - INFO - --- [CAMS] 开始为运行周期 2025-08-09 00:00 UTC 下载数据 ---
2025-08-09 22:33:29,697 - INFO - 将为 CAMS 运行周期下载 2 个特定预报时效的数据: ['21', '35']
2025-08-09 22:33:30,708 INFO [2024-09-26T00:00:00] Watch our [Forum]( https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-08-09 22:33:30,708 - INFO - [2024-09-26T00:00:00] Watch our [Forum]( https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-08-09 22:33:30,709 - INFO - 正在向 Copernicus ADS 发送请求以下载区域数据...
2025-08-09 22:33:32,101 INFO Request ID is 09cb73d4-a718-4156-abd1-312278b4ca6a
2025-08-09 22:33:32,101 - INFO - Request ID is 09cb73d4-a718-4156-abd1-312278b4ca6a
2025-08-09 22:33:32,456 INFO status has been updated to accepted
2025-08-09 22:33:32,456 - INFO - status has been updated to accepted
2025-08-09 22:33:42,297 INFO status has been

bcca1b4d4ed6dbdfd48f4704ebdb378b.nc:   0%|          | 0.00/159k [00:00<?, ?B/s]

2025-08-09 22:33:59,602 - INFO - ✅ 临时文件已成功下载到: C:\Users\zhang\Documents\Code\chromasky-toolkit\src\data\raw\cams_aod\20250809_t00z\temp_download
2025-08-09 22:33:59,603 - INFO - 检测到下载文件为 NetCDF，直接重命名。
2025-08-09 22:33:59,606 - INFO - ✅ AOD 数据清单已成功写入: C:\Users\zhang\Documents\Code\chromasky-toolkit\src\data\raw\cams_aod\20250809_t00z\manifest_aod.json
2025-08-09 22:33:59,608 - INFO - 
🎉 CAMS AOD 预报数据下载任务成功完成！
